<a href="https://colab.research.google.com/github/jacobeturpin/CXR14/blob/master/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

## Introduction

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

*Random subset provided [here](https://www.kaggle.com/nih-chest-xrays/sample)*

## Setup

In [0]:
%tensorflow_version 2.x
!pip install keras --upgrade --quiet
!pip install efficientnet --quiet

In [0]:
import gzip
import os
import time
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten
from keras.metrics import AUC
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import efficientnet.keras as efn

In [0]:
DATA_PATH = './drive/My Drive/CRX14/sample/'
EPOCHS = 50  # TODO: return to 50+ once model comparison work is complete
BATCH_SIZE = 32
CLASSES = [
  'Hernia',
  'Pneumonia',
  'Fibrosis',
  'Edema',
  'Emphysema',
  'Cardiomegaly',
  'Pleural_Thickening',
  'Consolidation',
  'Pneumothorax',
  'Mass',
  'Nodule',
  'Atelectasis',
  'Effusion',
  'Infiltration',
  'No Finding'
]
CHECKPOINT_DIR = './models/'

## Data Loading

### Full Dataset

In [0]:
def batch_download(first_n=None):

  # URLs for the zip files
  links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
    'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
    'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
    'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
  ]

  first_n = first_n or len(links)

  if first_n > len(links):
    raise('Number of files requested exceeds amount available')

  for idx, link in enumerate(links[:first_n]):
      fn = 'images_%02d.tar.gz' % (idx+1)
      print('downloading', fn, '...')
      urlretrieve(link, fn)  # download the zip file
  
  print("Download complete. Please check the checksums")

In [0]:
#batch_download(1)

In [0]:
def extract_gzip(filename):
  f = gzip.open(filename, 'rb')
  file_content = f.read()
  # TODO: implement save decompressed
  f.close()

In [0]:
for fn in os.listdir():
  if '.tar.gz' in fn:
    extract_gzip(fn)

### Sample Dataset

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
df = pd.read_csv(DATA_PATH + 'sample/sample_labels.csv')
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


In [83]:
df['Finding Labels'] = df['Finding Labels'].apply(lambda s: s.split('|'))
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,"[Emphysema, Infiltration, Pleural_Thickening, ...",5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,"[Cardiomegaly, Emphysema]",26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,[No Finding],1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,[Atelectasis],1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,"[Cardiomegaly, Edema, Effusion]",1,32,055Y,F,AP,2500,2048,0.168,0.168


In [84]:
# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Finding Labels')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000013_005.png,5,13,060Y,M,AP,3056,2544,0.139,0.139,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1
1,00000013_026.png,26,13,057Y,M,AP,2500,2048,0.168,0.168,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000017_001.png,1,17,077Y,M,AP,2500,2048,0.168,0.168,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,00000030_001.png,1,30,079Y,M,PA,2992,2991,0.143,0.143,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00000032_001.png,1,32,055Y,F,AP,2500,2048,0.168,0.168,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0


In [85]:
df = df[['Image Index'] + CLASSES]
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration,No Finding
0,00000013_005.png,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0
1,00000013_026.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2,00000017_001.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,00000030_001.png,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,00000032_001.png,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0


In [86]:
df[CLASSES].sum()

Hernia                  13
Pneumonia               62
Fibrosis                84
Edema                  118
Emphysema              127
Cardiomegaly           141
Pleural_Thickening     176
Consolidation          226
Pneumothorax           271
Mass                   284
Nodule                 313
Atelectasis            508
Effusion               644
Infiltration           967
No Finding            3044
dtype: int64

## Data Preparation

The ChestX-ray14 dataset is too large to fit entirely in memory when training; therefore, it's incrementally loaded via generator to reduce memory overhead. This is achieved using the Keras [Image Proprocessing](https://keras.io/preprocessing/image/) submodule.

In [87]:
# https://datascience.stackexchange.com/a/17445/91316

train_df, test_df = train_test_split(df, test_size=0.2)

print('Training/Validation Samples:  {}'.format(len(train_df)))
print('Test Samples:  {}'.format(len(test_df)))

Training/Validation Samples:  4484
Test Samples:  1122


In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [89]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    subset='training',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=CLASSES,
    target_size=(224, 224)
)

Found 3363 validated image filenames.


In [90]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 1121 validated image filenames.


In [91]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=DATA_PATH + 'sample/images/',
    x_col='Image Index',
    y_col=CLASSES,
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 1122 validated image filenames.


## Modeling

In [0]:
if not os.path.exists(CHECKPOINT_DIR):
  os.makedirs(CHECKPOINT_DIR)

In [0]:
class TimeHistory(keras.callbacks.Callback):
  """Object used on keras callbacks to measure epoch training time
  
  Args:
    None

  Params:
    time (list): collection of times in seconds for each epoch's training
  
  """

  def on_train_begin(self, logs={}):
    self.times = []

  def on_epoch_begin(self, batch, logs={}):
    self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
    self.times.append(time.time() - self.epoch_time_start)

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

A pre-built ResNet model from the Keras library is used. Documentation on the model can be found [here](https://keras.io/applications/). Pre-trained weights from the ImageNet dataset are used.

In [94]:
resnet_base = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(resnet_base.output)

resnet = Model(input=resnet_base.input, outputs=output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [95]:
resnet.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
resnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', AUC()]
)

In [97]:
resnet_time = TimeHistory()
resnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
resnet_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR + 'resnet-{epoch:02d}.hdf5', period=5)

resnet_history = resnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[resnet_time, resnet_stopping, resnet_checkpoint]
)

Epoch 1/50
106/106 [==============================] - 95s 898ms/step - loss: 0.2340 - binary_accuracy: 0.9212 - auc_3: 0.8205 - val_loss: 0.5640 - val_binary_accuracy: 0.7109 - val_auc_3: 0.7053
Epoch 2/50
106/106 [==============================] - 76s 713ms/step - loss: 0.2052 - binary_accuracy: 0.9293 - auc_3: 0.8600 - val_loss: 0.2442 - val_binary_accuracy: 0.8717 - val_auc_3: 0.8026
Epoch 3/50
106/106 [==============================] - 75s 712ms/step - loss: 0.1988 - binary_accuracy: 0.9296 - auc_3: 0.8679 - val_loss: 0.3175 - val_binary_accuracy: 0.8106 - val_auc_3: 0.7308
Epoch 4/50
106/106 [==============================] - 75s 711ms/step - loss: 0.1942 - binary_accuracy: 0.9319 - auc_3: 0.8803 - val_loss: 0.2569 - val_binary_accuracy: 0.8636 - val_auc_3: 0.7645
Epoch 5/50
106/106 [==============================] - 77s 722ms/step - loss: 0.1878 - binary_accuracy: 0.9329 - auc_3: 0.8888 - val_loss: 0.0559 - val_binary_accuracy: 0.9250 - val_auc_3: 0.8062
Epoch 6/50
106/106 [=====

### DenseNet

In [98]:
densenet_base = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(densenet_base.output)

densenet = Model(input=densenet_base.input, outputs=output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [99]:
densenet.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
densenet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', AUC()]
)

In [101]:
densenet_time = TimeHistory()
densenet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
densenet_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR + 'resnet-{epoch:02d}.hdf5', period=5)

densenet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[densenet_time, densenet_stopping, densenet_checkpoint]
)

Epoch 1/50
106/106 [==============================] - 126s 1s/step - loss: 0.2287 - binary_accuracy: 0.9178 - auc_4: 0.8257 - val_loss: 0.0278 - val_binary_accuracy: 0.9250 - val_auc_4: 0.8216
Epoch 2/50
106/106 [==============================] - 82s 774ms/step - loss: 0.2034 - binary_accuracy: 0.9294 - auc_4: 0.8592 - val_loss: 0.0076 - val_binary_accuracy: 0.9254 - val_auc_4: 0.8539
Epoch 3/50
106/106 [==============================] - 82s 771ms/step - loss: 0.2015 - binary_accuracy: 0.9297 - auc_4: 0.8662 - val_loss: 0.2934 - val_binary_accuracy: 0.9321 - val_auc_4: 0.8526
Epoch 4/50
106/106 [==============================] - 82s 775ms/step - loss: 0.1961 - binary_accuracy: 0.9305 - auc_4: 0.8734 - val_loss: 0.3293 - val_binary_accuracy: 0.9282 - val_auc_4: 0.8329
Epoch 5/50
106/106 [==============================] - 82s 776ms/step - loss: 0.1926 - binary_accuracy: 0.9311 - auc_4: 0.8811 - val_loss: 0.4044 - val_binary_accuracy: 0.9269 - val_auc_4: 0.8340
Epoch 6/50
106/106 [=======

### EfficientNet

EfficientNet is a lightweight CNN architecture that is designed to require significantly less compute than other state of the art architectures on common transfer learning datasets.

Pre-built EfficientNet models built in Keras are used from the efficientnet library available on [GitHub](https://github.com/qubvel/efficientnet) and installable via PyPI.

In [102]:
efficientnet_base = efn.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(efficientnet_base.output)

efficientnet = Model(input=efficientnet_base.input, outputs=output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [103]:
efficientnet.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 48) 1296        input_7[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [0]:
efficientnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', AUC()]
)

In [105]:
efficientnet_time = TimeHistory()
efficientnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
efficientnet_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR + 'resnet-{epoch:02d}.hdf5', period=5)

efficientnet_history = efficientnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[efficientnet_time, efficientnet_stopping, efficientnet_checkpoint]
)

Epoch 1/50
106/106 [==============================] - 147s 1s/step - loss: 0.2352 - binary_accuracy: 0.9181 - auc_5: 0.8125 - val_loss: 0.2693 - val_binary_accuracy: 0.9296 - val_auc_5: 0.8386
Epoch 2/50
106/106 [==============================] - 97s 917ms/step - loss: 0.2027 - binary_accuracy: 0.9308 - auc_5: 0.8640 - val_loss: 0.4368 - val_binary_accuracy: 0.9268 - val_auc_5: 0.8556
Epoch 3/50
106/106 [==============================] - 97s 916ms/step - loss: 0.1909 - binary_accuracy: 0.9332 - auc_5: 0.8834 - val_loss: 0.2964 - val_binary_accuracy: 0.9278 - val_auc_5: 0.8582
Epoch 4/50
106/106 [==============================] - 97s 917ms/step - loss: 0.1746 - binary_accuracy: 0.9373 - auc_5: 0.9039 - val_loss: 0.3044 - val_binary_accuracy: 0.9255 - val_auc_5: 0.8610
Epoch 5/50
106/106 [==============================] - 97s 914ms/step - loss: 0.1590 - binary_accuracy: 0.9421 - auc_5: 0.9237 - val_loss: 0.0390 - val_binary_accuracy: 0.9301 - val_auc_5: 0.8376
Epoch 6/50
106/106 [=======

## Results

In [0]:
resnet_pred = resnet.predict_generator(
    generator=test_generator
)

In [107]:
resnet_pred.shape
resnet_pred[:,0].shape

(1122,)

In [108]:
from sklearn.metrics import roc_auc_score

for idx, cls in enumerate(CLASSES):
  print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], resnet_pred[:,idx]))

Hernia AUC:   0.46112600536193027
Pneumonia AUC:   0.4186851211072664
Fibrosis AUC:   0.5691077875193398
Edema AUC:   0.46083638583638586
Emphysema AUC:   0.4520417422867513
Cardiomegaly AUC:   0.46935336033825137
Pleural_Thickening AUC:   0.5396215596330275
Consolidation AUC:   0.6046511627906977
Pneumothorax AUC:   0.5563982746225737
Mass AUC:   0.46910355295903416
Nodule AUC:   0.49770810258157955
Atelectasis AUC:   0.4402584632673127
Effusion AUC:   0.42782345335971483
Infiltration AUC:   0.5304258079735786
No Finding AUC:   0.5323588137825824


## Conclusions